In [1]:
import numpy as np
import pandas as pd
import string
from scipy.stats import boxcox
from collections import defaultdict, Counter
import xgboost as xgb

/home/rx78rx91/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path='/home/train.json'
test_path='/home/test.json'

list_room=pd.read_json(path)
list_test=pd.read_json(test_path)
ntrain = list_room.shape[0]
#Fix & get label、test_set_ID

listing_id = list_test.listing_id.values
y_map = {'low': 2, 'medium': 1, 'high': 0}
list_room['interest_level'] = list_room['interest_level'].apply(lambda x: y_map[x])
y_train = list_room.interest_level.values
train = list_room.drop(['listing_id', 'interest_level'], axis=1)
test = list_test.drop('listing_id', axis=1)
#fix_outlierValue in Testset
fix_bathrooms=list_test.bathrooms.tolist()
for i in range(len(fix_bathrooms)):
    if fix_bathrooms[i] == 20.0:
        fix_bathrooms[i]= 2.0                
    elif fix_bathrooms[i]==112.0:
        fix_bathrooms[i]= 1.5
list_test["bathrooms"]=fix_bathrooms
#merge
train_test = pd.concat((train, test), axis=0).reset_index(drop=True)
#Customers may interest on price–performance ratio
train_test["cp_bathrooms"]=train_test.price.divide(train_test.bathrooms,axis=0)
train_test["cp_bedrooms"]=train_test.price.divide(train_test.bedrooms,axis=0)
train_test["bed_bath_sum"]=train_test.bathrooms.add(train_test.bedrooms,axis=0)
train_test["cp_bed_bath_sum"]=train_test.price.divide(train_test.bed_bath_sum,axis=0)
train_test["bed_bath_sub"]=train_test.bathrooms.subtract(train_test.bedrooms,axis=0)
train_test["cp_bed_bath_su"]=train_test.bathrooms.divide(train_test.bedrooms,axis=0)
#filtered dict with less than times counts
def feature_filter(feature_map,times):
    feature_map_filtered={}
    index=0
    for i in feature_map:
        value=feature_map[i][1]
        
        
        if value > times:
           
            feature_map_filtered.update({i:index})
            index+=1
    return feature_map_filtered


In [3]:
#rename df into Sequential label and count each label
def re_label_and_counts(dataframe):
    #return list,dict_info
    list_data=dataframe.tolist()
    label_dict={}
    index=1
    for list_data_index in range(len(list_data)):
        if list_data[list_data_index] not in label_dict:
            label_dict.update({list_data[list_data_index]:1})
            
            #label_dict.update({list_data[list_data_index]:[index,1]})
            #index+=1 
            #list_data[list_data_index]=index            
        else:
            dict_info=label_dict[list_data[list_data_index]]
            label_dict.update({list_data[list_data_index]:dict_info+1})
            #dict_info=dict_info+1
            
            #dict_info[1]=dict_info[1]+1
            #label_dict.update({list_data[list_data_index]:dict_info})
            #list_data[list_data_index]=dict_info[0]
    
    
    

    f_score_df=pd.DataFrame.from_dict(label_dict,orient="index")
    
    f_score_df.columns=["counts"]
    f_score_df.sort_values(by=["counts"],inplace=True ,ascending=0)
    return f_score_df   
def tf_Df_generator(dict_df,target_df):
    target_list=target_df.tolist()
    index=list(range(0,len(dict_df)))
    dict_df["index"]=index
    
    dict_dict=dict_df.T.to_dict('list')
    
    feature_length=len(dict_dict)
    customers_length=len(train_test)
    New_columns=np.zeros([customers_length,feature_length])
    for list_index in range(len(target_list)):
        if target_list[list_index] in dict_dict:
            New_columns[list_index][dict_dict[target_list[list_index]][1]]=1
    tf_df=pd.DataFrame(New_columns,columns=dict_dict.keys())
    
    return tf_df
            
    

In [4]:
#clean building_id
building_df=re_label_and_counts(train_test.building_id)
building_df_top50=building_df[0:50]
building_table_tf=tf_Df_generator(building_df_top50,train_test.building_id)
train_test = pd.concat((train_test, building_table_tf), axis=1).reset_index(drop=True)

/home/rx78rx91/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
#create time parameters

train_test['Date'] = pd.to_datetime(train_test['created'])
train_test['month'] = train_test['Date'].dt.month
train_test['day'] = train_test['Date'].dt.day
train_test['wday'] = train_test['Date'].dt.dayofweek
train_test['yday'] = train_test['Date'].dt.dayofyear
train_test['hour'] = train_test['Date'].dt.hour
#train_test['Year'] = train_test['Date'].dt.year

In [6]:
#description

train_test['desc'] = train_test['description']
train_test['desc'] = train_test['desc'].apply(lambda s: s.replace('<p><a  website_redacted ', ''))
train_test['desc'] = train_test['desc'].apply(lambda s: s.replace('!<br /><br />', ''))
string.punctuation.__add__('!!')
string.punctuation.__add__('(')
string.punctuation.__add__(')')
train_test['desc'] = train_test['desc'].apply(lambda s:''.join([l for l in s if l not in string.punctuation]))
train_test['desc_letters_count'] = train_test['description'].apply(lambda s: len(s.strip()))
train_test['desc_words_count'] = train_test['desc'].apply(lambda s: 0 if len(s.strip()) == 0 else len(s.split(' ')))
train_test.drop(['description', 'desc'], axis=1, inplace=True)


In [7]:
#covert display_address into format
train_test['address1'] = train_test['display_address']
train_test['address1'] = train_test['address1'].apply(lambda s: s.lower())

address_dictMap = {
    'w': 'west',
    'st.': 'street',
    'ave': 'avenue',
    'st': 'street',
    'e': 'east',
    'n': 'north',
    's': 'south'
}
#bc_price, tmp = boxcox(train_test.price)
#train_test['bc_price'] = bc_price

def address_reformat(s):
    s = s.split(' ')
    out = []
    for x in s:
        if x in address_dictMap:
            out.append(address_dictMap[x])
        else:
            out.append(x)
    return ' '.join(out)


train_test['address1'] = train_test['address1'].apply(lambda s:''.join([l for l in s if l not in string.punctuation]))
train_test['address1'] = train_test['address1'].apply(lambda s: address_reformat(s))

new_cols = ['street', 'avenue', 'east', 'west', 'north', 'south']

for col in new_cols:
    train_test[col] = train_test['address1'].apply(lambda s: 1 if col in s else 0)

train_test['other_address'] = train_test[new_cols].apply(lambda s: 1 if s.sum() == 0 else 0, axis=1)


address1_df=re_label_and_counts(train_test.address1)
address1_df_top50=address1_df[0:100]
address1_table_tf=tf_Df_generator(address1_df_top50,train_test.address1)
train_test = pd.concat((train_test, address1_table_tf), axis=1).reset_index(drop=True)

# photos counts may repreasent the quality of house
train_test['photos_count'] = train_test['photos'].apply(lambda x: len(x))

/home/rx78rx91/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
#convert  features into hash key
def clean(s):
    s="".join(l for l in s if l not in string.punctuation)       
    x = s.replace(" ", "")
    x = x.replace("twenty four hour", "24")
    x = x.replace("24/7", "24")
    x = x.replace("247", "24")
    x = x.replace("24hr", "24")
    x = x.replace("24hour", "24")
    x = x.replace("24hour", "24")
    x = x.replace("24 hour", "24")
    x = x.replace("common", "cm")
    x = x.replace("concierge", "doorman")
    x = x.replace("bicycle", "bike")
    x = x.replace("private", "pv")
    x = x.replace("deco", "dc")
    x = x.replace("decorative", "dc")
    x = x.replace("onsite", "os")
    x = x.replace("outdoor", "od")
    x = x.replace("ss appliances", "stainless")
    x=x.replace("24fulltime","24do")
    x=x.replace("allpetsok","pets")    
    #x=x.replace("cats","pets")
    #x=x.replace("dogs","pets")
    return x

def feature_hash(x):
    cleaned = clean(x)
    key = cleaned[:4].strip()    
    return key,cleaned

#convert string to lower
features = train_test[["features"]].apply(lambda _: [list(map(str.strip, map(str.lower, x))) for x in _])


def hash_transform():    
    features_counts=[]
    feature_map={}
    count=0
    for feature in features.features:
        features_counts.append(len(feature))
        for single_feature in feature:
            key,new_string=feature_hash(single_feature) 
            if key not in feature_map:
                feature_map.update({key:[count,1,[new_string]]})
                count+=1
            else:
                key_word_info=feature_map[key]

                key_word_info[1]+=1
                if new_string not in key_word_info[2]:
                    key_word_info[2].append(new_string)
    return feature_map,features_counts

feature_map,features_counts=hash_transform()

feature_map_filtered=feature_filter(feature_map,50)
    

In [9]:
#generate TF feature_table by hash Key   
def gen_feature_table(feature_map_filtered): 
    feature_length=len(feature_map_filtered)
    customers_length=len(train_test)
    New_columns=np.zeros([customers_length,feature_length])
    index=0
    for feature in features.features:
        for single_feature in feature:
            key,new_string=feature_hash(single_feature)

            if key in feature_map_filtered:

                New_columns[index][feature_map_filtered[key]]=1
        index+=1        
    feature_df=pd.DataFrame(New_columns,columns=list(feature_map_filtered.keys()))
    return feature_df

feature_df=gen_feature_table(feature_map_filtered)            


In [10]:
#clean building_id
manager_df=re_label_and_counts(train_test.manager_id)
manager_df_top50=manager_df[0:50]
manager_table_tf=tf_Df_generator(manager_df_top50,train_test.manager_id)
train_test = pd.concat((train_test, manager_table_tf), axis=1).reset_index(drop=True)

/home/rx78rx91/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
def xgb_feature_filter(x_train,y_train):
    NFOLDS = 5

    params = {
        'eta':.01,
        'colsample_bytree':.8,
        'subsample':.8,
        'seed':0,
        'nthread':8,
        'objective':'multi:softprob',
        'eval_metric':'mlogloss',
        'num_class':3,
        'silent':1,
        'max_depth':5,
        'nfold':5

    }


    dtrain = xgb.DMatrix(data=x_train, label=y_train)
    bst = xgb.train(params, dtrain, 2000)
    f_score=bst.get_fscore()
    f_score_df=pd.DataFrame.from_dict(f_score,orient="index")
    f_score_df.columns=["counts"]
    f_score_df.sort(["counts"],inplace=True ,ascending=0)
    f_score_df["features"]=f_score_df.index
    return f_score_df

In [12]:
f_score=xgb_feature_filter(feature_df[:ntrain],y_train)

/home/rx78rx91/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [13]:

drop_col=['display_address','photos','features','street_address','Date', 'created','building_id','manager_id','address1']
train_test.drop(drop_col,axis=1,inplace=True)

f_top50=f_score.features[0:50].tolist()
top_feature_df=feature_df[:ntrain][f_top50]
x_train=train_test[:ntrain]

train_data = pd.concat((x_train, top_feature_df), axis=1).reset_index(drop=True)
#train_data.to_csv("D:\\test\\f_room_train.csv",index=False)
y_df=pd.DataFrame({"y":y_train})
y_df.to_csv("D:\\test\\f_room_lebel.csv",index=False)
x_test=train_test[ntrain:]
x_test.reset_index(inplace=True)
top_test_feature_df=feature_df[ntrain:][f_top50].reset_index(drop=True)
test_data = pd.concat((x_test, top_test_feature_df), axis=1).reset_index(drop=True)
test_data["listing_id"]=list_test.listing_id.values
#test_data.to_csv("D:\\test\\f_room_test.csv",index=False)

In [14]:
def xgb_train(x_train,y_train):
    NFOLDS = 5

    params = {
        'eta':.02,
        'colsample_bytree':.7,
        'subsample':.7,
        'seed':321,
        'nthread':8,
        'objective':'multi:softprob',
        'eval_metric':'mlogloss',
        'num_class':3,
        'silent':1,
        'max_depth':10,
        'min_child_weight' : 1
        
        
        

    }


    dtrain = xgb.DMatrix(data=x_train, label=y_train)
    bst = xgb.train(params, dtrain, 2000)
    
    return bst

In [15]:
bst=xgb_train(train_data,y_train)

In [16]:
x_test=test_data
listing_id=x_test.listing_id
x_test.drop(["listing_id","index"],axis=1,inplace=True)
dtest = xgb.DMatrix(data=x_test)
pred=bst.predict(dtest)
preds = pd.DataFrame(pred)

cols = ['high', 'medium', 'low']

preds.columns = cols

preds['listing_id'] = listing_id

preds.to_csv("/home/result.csv", index=None)